# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [7]:
from unityagents import UnityEnvironment
import numpy as np
from ddpg_agent import DDPGAgent
import torch

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [12]:
def run(episodes):
    device = 'cpu'
    for episode in range(episodes):
        done = False
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations 

        while not done:
#             states = torch.from_numpy(states).float().unsqueeze(0).to(device)
            actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
            actions = agent.act(states)
            print(actions)
            actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
#             rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
#             scores += env_info.rewards                         # update the score (for each agent)
            states = next_states  

In [13]:
run(1)

[[-0.27526722 -0.11589276  2.16242327 -0.49904821]]
[[ 1.22045332 -1.61512503 -0.75330612  0.02710718]]
[[ 0.25869279 -1.51806799 -0.52099326  1.34234344]]
[[-0.23838993  1.75259181  0.72609987  0.50029526]]
[[ 1.07508069 -0.97563573 -0.78714293 -0.51712498]]
[[-0.82932268 -0.00184588 -1.27420944 -0.50780528]]
[[ 0.96512082 -0.63408661  0.48596824 -1.17131614]]
[[ 0.6949632   0.18024489  0.69328768 -0.41284653]]
[[-1.40040785 -1.88594929  1.9151777  -0.10097587]]
[[ 1.08497721 -1.79187467 -0.10926844 -0.11131852]]
[[ 1.674461    1.20001802 -0.05578128  0.45238343]]
[[-1.21251024 -1.07037651 -0.12850564 -1.05158086]]
[[0.42750373 0.11483528 0.08200035 0.42070252]]
[[0.84422487 0.91445651 0.97986658 0.84654167]]
[[ 0.28704544 -0.43077555  0.5810651   1.27925712]]
[[ 2.02113391  0.78418718 -0.2146388  -0.82245151]]
[[1.54237045 1.7998524  1.42358886 0.06944443]]
[[0.65355334 1.00832111 0.85419699 1.37205977]]
[[-0.06030262 -2.23102618 -1.49199062  2.62501116]]
[[ 0.99607447  0.27949715 -1

[[-0.82721079 -0.5314781   1.41190802  0.01958915]]
[[-1.09381197 -0.36831803  1.56861395 -0.84133883]]
[[ 0.32949346 -0.06721504 -1.26498006 -0.07708449]]
[[ 0.90095861 -1.23752476 -1.26603942 -0.62319961]]
[[ 0.21472685  1.19720671 -1.99295172 -0.20241362]]
[[-0.91406385 -0.77265727  2.17932513 -2.64131418]]
[[-0.04212563  1.57092979  1.45289558 -0.24762933]]
[[-0.62631193 -0.42017745 -0.28782094 -0.21845349]]
[[ 0.80340015 -0.03539476  0.52769616  0.46196456]]
[[ 0.59213337 -1.82816249 -0.06143625 -1.67222995]]
[[-1.11471388  1.58775291  0.58100189  1.03050441]]
[[ 0.93122876 -0.60821959  0.59447405  1.18263965]]
[[-0.84690997  1.19531167 -0.49987771  0.99008946]]
[[-1.30046211 -0.56764806 -1.77995058  0.65790809]]
[[-0.2075886   0.25724436 -0.16725109 -0.13862268]]
[[ 1.02673829 -0.72683819  0.33624439  0.43038578]]
[[-1.00834383 -0.56070517  0.04553415 -1.53559645]]
[[ 1.53945253 -1.07759913 -0.37100505 -0.85863019]]
[[-1.23054375 -0.34770258  0.39636965 -1.17318281]]
[[-0.4880417

[[-0.03461443 -0.05633486 -2.59031654  0.77549116]]
[[ 0.216838   -1.14712677  0.43326201 -0.89021819]]
[[ 1.01280318 -0.58993291  0.57048616  0.78715963]]
[[0.09011408 0.45745769 0.14734885 1.60989849]]
[[-1.01122528 -0.31532441  1.23366978  0.49254865]]
[[-0.3620052   0.49504438 -1.2437561  -0.17370967]]
[[-0.08240985 -1.21220168 -0.24371973 -1.47395177]]
[[ 0.51957376 -0.13706838 -0.28599382 -0.37035433]]
[[ 1.28415315  0.58586547 -1.01313577  0.29206651]]
[[-0.5869584  -0.89410245 -0.07447003  2.14945071]]
[[ 0.30515878  0.0961497  -0.46686213  0.17678773]]
[[ 0.03017921  0.07602809 -1.50691078  0.40994679]]
[[-0.04627931 -0.45496835  0.49935106 -0.09935899]]
[[-0.68701893 -0.53978688 -1.59976527 -0.88794949]]
[[ 1.52761247 -1.04004481 -0.34058258  0.32144318]]
[[-1.228208    1.96041007 -0.19468953  1.1933083 ]]
[[-0.11954737  0.29828572 -1.81423784 -0.32104615]]
[[ 0.14271832  1.62246181 -0.28935331 -0.33644745]]
[[-0.29294512 -0.26819794  0.01152204  0.04181519]]
[[-0.40053903  0

[[-0.03645644  1.44730905  0.10435896  0.53466785]]
[[ 0.32616718  0.0743282   0.6904923  -1.90052602]]
[[-1.57233584  2.05023184 -0.50300975  0.33647122]]
[[ 0.31398264 -1.2025984  -1.48487233 -0.01132492]]
[[-1.62837531  0.82984161 -0.26859752  0.32472419]]
[[-2.58840118  1.13352987  0.21293407 -1.81660608]]
[[ 0.22686083 -0.6872186  -1.84322493 -0.20641001]]
[[1.31156142 1.91559054 1.46092513 0.45056493]]
[[-0.26107895 -0.67511191  0.37159851  0.026246  ]]
[[-1.19429427  0.61455557  1.26039396  0.59216729]]
[[-0.35419966  0.84640672  1.21887176 -0.06238606]]
[[-0.80939174  0.08838204  0.42536108 -0.84432324]]
[[-0.40555861 -0.42083797  1.57840665  0.76638064]]
[[-0.38800605 -0.92042551 -1.46027184  0.48701092]]
[[0.95560722 0.66627687 0.18486786 0.19605346]]
[[-0.30223322  0.24109112 -0.03527762  0.03711543]]
[[-0.38539228 -0.15622171 -0.78569776 -1.12883102]]
[[-0.35192677  0.70643062 -1.63095244  1.08040167]]
[[-0.39050046 -0.41956733 -1.23715488 -0.31145389]]
[[ 1.23608301  1.415

[[ 0.35699593 -2.01387818 -1.86338495 -0.44144038]]
[[ 0.25844408  0.98627816 -0.59156759 -0.30092748]]
[[0.95989635 1.96804844 0.53766826 2.79540758]]
[[ 0.60420389  0.51783467 -0.36190635  1.74923157]]
[[ 0.38040581 -0.67441222  0.45268773 -0.17531614]]
[[0.13668455 0.10683767 0.88672727 0.47317959]]
[[ 0.08746512  0.39043382 -0.54878382 -0.68922637]]
[[ 0.45275775 -1.02384031  1.16995402  0.43525256]]
[[-0.28943812  1.07569227  1.50010121  0.71773926]]
[[-1.24550419  0.71719113  0.37961059  0.6664618 ]]
[[ 0.16755192 -0.41179572 -0.04548682  0.16476273]]
[[-1.52131222  0.7210391   0.47647365 -0.03498913]]
[[0.49172707 0.02767593 1.26679284 0.24372411]]
[[-0.38241271  0.05656078 -0.951385   -0.11901826]]
[[ 0.01444863  2.73475502 -1.07182527  0.70721884]]
[[ 0.96056629 -0.59466796 -0.50640867  0.09334365]]
[[ 0.29125906  0.1135048  -0.26197953  0.29426313]]
[[ 0.14873856 -0.80140255 -0.84533517  1.66485486]]
[[-0.73944527 -0.30248664 -0.05918223 -0.50132545]]
[[ 0.32226102  0.9731510

...

[[-0.37127147 -0.18907309  0.2009603  -1.64165937]]
[[-0.11747463  0.79851909  1.53385698  1.14993246]]
[[ 0.50332371 -0.46503583 -0.74618862  0.83714952]]
[[ 1.80541894  0.3179005   0.94169786 -0.7279727 ]]
[[ 0.38277205 -0.32988891 -1.03754386 -1.99962333]]
[[ 0.382187    1.70655701  0.59603398 -0.43532606]]
[[-0.82676989  0.60996134  3.57665523 -1.32009564]]
[[-0.71689759  1.11906807 -0.66668456  1.86138857]]
[[-1.4663779  -1.44407016 -0.18603318  0.36502184]]
[[-0.58682772 -1.41662649 -1.22473608 -0.05148811]]
[[ 0.9902812  -0.28684903  0.63254257 -1.00771937]]
[[ 2.33841621 -0.08466289 -1.30273458  0.14957485]]
[[ 0.55970522  0.56617656 -0.46490884  0.06939512]]
[[ 1.62024023  2.03078711 -0.32525816 -1.87723659]]
[[ 0.38708975 -1.44150215  0.85874329 -1.1536241 ]]
[[ 0.11656618 -0.31567367 -1.16176002 -1.31010626]]
[[-1.02933433  1.76092662  0.32067945  0.21383304]]
[[-0.90196836 -0.87411201  0.1792507  -1.79906733]]
[[-0.14561117 -2.01467995 -1.12993021 -1.69665618]]
[[ 0.1654794

[[-0.97067785 -1.13368995  0.78139035  0.66620597]]
[[1.1867741  0.40275756 1.02237117 0.78579516]]
[[ 0.32159586 -1.16917152 -1.11910972 -1.4118909 ]]
[[-0.81018766  0.14842133 -0.37846864 -1.34253368]]
[[-0.09653977  0.22319535 -0.27533806 -2.58918439]]
[[-1.21644499 -0.29104011 -0.3726886  -0.0663532 ]]
[[ 1.07728776 -1.17113654 -2.42573959  0.333881  ]]
[[ 1.18697832  0.30971654 -1.17681285 -0.76875841]]
[[-0.50886434  0.4538652   0.0603624   0.31072964]]
[[ 0.00271036 -0.96844998 -2.14114056  1.62974836]]
[[ 0.66371322 -1.29988069  0.98279428  1.60785751]]
[[1.47218487 0.70494794 0.38603133 0.59301665]]
[[-0.63373126 -1.43287555 -0.20283833  0.188037  ]]
[[-0.61350874  0.1160788   0.18522999  1.36541056]]
[[-1.19148787  0.88859708  0.12909372  0.09257265]]
[[ 0.38783567 -1.03386681  1.32122258  1.33226236]]
[[ 0.34565329 -0.60603342 -1.53279887  0.96737997]]
[[ 0.13145678  2.06633838  0.03698221 -0.99406428]]
[[-0.48362207 -0.72119075 -0.91119626 -0.63696909]]
[[ 0.19633075 -0.706

[[1.05722035 0.73099495 1.42847128 1.08892662]]
[[-3.04705063  1.53104249  0.34970044 -0.8445991 ]]
[[-0.54954499  0.80670387 -2.22792128 -0.8890317 ]]
[[-1.07756739  0.3015797   0.37007425  1.20408828]]
[[ 0.14935997 -0.25467624  1.42171746 -1.08583859]]
[[-0.03109848  0.66443013  0.19311204  0.67926509]]
[[-1.67101556  1.1365603   0.62161385  2.5654227 ]]
[[ 1.25425241e+00  1.49030314e+00 -1.07980708e-03  1.12867593e+00]]
[[ 0.37645058 -0.45347661  0.70067674 -0.55004551]]
[[-0.57444639  1.39118989 -2.31390689  0.17194816]]
[[-1.31677003 -0.39464516 -0.36904704 -1.30407277]]
[[0.1142871  0.74891839 0.11892419 1.39418922]]
[[ 1.9471116   1.67423914 -1.9170061   0.68668598]]
[[-0.04165071  0.15824961 -0.47693383 -0.54806066]]
[[ 1.22339377  1.41538965  0.06632118 -0.28005877]]
[[ 0.56968893 -1.36829329  0.37572745  1.59104912]]
[[ 1.31479883 -0.72143655  0.14996964 -1.06473663]]
[[ 1.30842425  0.12872736  0.2305387  -0.49742237]]
[[ 1.3004909  -0.28934107 -0.43688513 -0.92237048]]
[[-0

[[ 1.01652103 -1.35153893 -1.31702807 -0.61443748]]
[[ 0.02796429  0.25260015 -0.95055168 -0.1890356 ]]
[[-0.29048789  0.14860893 -1.83811528  0.76604169]]
[[ 1.2534008   0.81452697  0.88314892 -0.49242669]]
[[-0.27866496  0.70752007  0.6832646   0.32333949]]
[[-0.09315358 -0.31368404 -0.51032084  0.14322151]]
[[-0.5286485  -1.83819337 -0.22589232 -1.09565778]]
[[-0.08739685 -0.08043863 -1.85730749 -0.81914542]]
[[ 1.09790828 -0.19875067 -1.02946961 -2.1851362 ]]
[[ 0.94908263 -0.87074242 -1.88043016  0.74843216]]
[[ 0.30758367  1.56599656 -1.12088085 -1.61436221]]
[[-0.03411763 -0.30373896 -0.3085489   1.28407817]]
[[ 0.74968513 -0.04766736 -0.10343716 -1.09435671]]
[[ 1.0378155  -1.334839   -0.26813945  0.80305084]]
[[0.10820074 0.92378585 0.78462424 1.00309566]]
[[-1.43514134  1.4514961  -0.37232776 -1.37375169]]
[[-0.8686849   0.90151738 -0.06252618 -0.93847852]]
[[ 0.01865521  0.90938474  1.02394798 -0.30862761]]
[[ 0.91343848 -0.92428389 -0.63312043 -1.19361323]]
[[ 1.6253585  -1

KeyboardInterrupt: 